**Test Area for Prescient Project on Druggability**

**Install dependencies in the terminal:**

In [ ]:
# pip install biopandas biopython pandas numpy

**Import libraries:**

In [ ]:
from biopandas.pdb import PandasPdb                            # To handle PDB files as dataframes
from Bio.PDB import PDBParser, PDBIO, Select, NeighborSearch   # To parse PDB files and handle structures
from Bio.PDB.SASA import ShrakeRupley                          # To compute solvent accessible surface area (SASA)
from Bio.SeqUtils import seq1                                  # To convert 3-letter to 1-letter amino acid codes
from Bio.SeqUtils.ProtParam import ProteinAnalysis             # To compute protein properties
import pandas as pd, tempfile                                  # To handle dataframes and temp files

**Configuration:**

Use two definitions (distance AND ΔSASA)

In [ ]:
PDB_ID = "5X8L"             # Structure from PDB: PD-L1 + atezolizumab Fab (Antigent Binding Fragment)
pdb_path = f"{PDB_ID}.pdb"  # Save filename as 5xxy.pdb
DIST_CUTOFF = 4.5           # If any heavy atom of a PD-L1 residue is ≤ 4.5 Å from any antibody atom, mark it as contacting
DSASA_THRESH = 2.0          # Call a residue an epitope if its surface buries at least 2 Å² upon binding
PH = 7.4                    # Side-chain charge depends on pH; 7.4 is physiological standard

**Download the structure:**

In [ ]:
ppdb = PandasPdb().fetch_pdb(PDB_ID)                                    # Use fetch_pdb to save the structure as a PandasPdb object
ppdb.to_pdb(path=pdb_path, records=None, gz=False, append_newline=True) # Write the PDB file to disk
print(f"Downloaded PDB structure {PDB_ID} to {pdb_path}")               # Confirm download

**Define chains:**

For 5XXY, chain A = PD-L1, chains H and L = antibody heavy and light

In [ ]:
pdl1_chains   = ['A']       # PD-L1 antigen
binder_chains = ['H','L']   # Antibody chains

**Load structure with Biopython:**

A PDB file is just a plain text file with atomic coordinates.

PDBParser is the tool that reads that text and builds a hierarchical data structure you can work with in Python.

In [ ]:
parser = PDBParser(QUIET=True)                      # Reads the coordinates into Python objects (Structure → Model → Chain → Residue → Atom).
structure = parser.get_structure(PDB_ID, pdb_path)  # Load structure from file
model = structure[0]                                # First (and only) model in PDB - usually one for x ray and many for NMR ensembles

**Helper functions:**

**residue_id** - Return unique ID for a residue (chain, number, insertion code)

**residue_features** - Hydrophobicity (Kyte–Doolittle) and net charge at pH


In [ ]:
def residue_id(res):
    return (res.get_parent().id, res.get_id()[1], res.get_id()[2].strip() or "")

# res.get_parent().id → finds the chain ID (e.g., chain "A" or "B" in a protein).
# res.get_id() → gives a tuple with information about the residue.
# [1] → the residue number (e.g. 45, meaning "residue #45").
# [2] → an insertion code (a letter used if two residues share the same number).
# .strip() or "" → removes extra spaces. If nothing’s left, it uses an empty string "".
# This ensures every residue has a unique identifier like: ("A", 45, "")

def residue_features(resname, pH=7.4): #resname comes from structure data i.e. TYR / LYS
    try:
        aa1 = seq1(resname.strip(), custom_map={'SEC':'U','PYL':'O'}) # Convert 3-letter to 1-letter code; handle special cases
        pa = ProteinAnalysis(aa1)                                     # Create ProteinAnalysis object that can compute properties of the amino acid
        return pa.gravy(), pa.charge_at_pH(pH)                        # gravy = hydrophobicity; charge_at_pH = net charge at given pH
    except Exception:
        return None, None


**Create PDL1 only structure:**

In [ ]:
class OnlyChains(Select):                                            # Helper to save only specified chains from a structure
    def __init__(self, keep): self.keep=set(keep)                    # Store chains to keep as a set for fast lookup
    def accept_chain(self, chain): return chain.id in self.keep      # Only accept chains in the keep set

pdbio = PDBIO()                                                      # Create PDBIO object to write structures to file
pdbio.set_structure(structure)                                       # Set the structure to write
pdl1_only_path = tempfile.gettempdir() + f"/{PDB_ID}_pdl1_only.pdb"  # Create a temporary file path for PD-L1 only structure i.e. /tmp/5XXY_pdl1_only.pdb
pdbio.save(pdl1_only_path, select=OnlyChains(pdl1_chains))           # Save only PD-L1 chains to the temporary file
print(f"Saved PD-L1 only structure to {pdl1_only_path}")             # Confirm save

pdl1_struct = parser.get_structure(PDB_ID+"_pdl1", pdl1_only_path)   # Load PD-L1 only structure into memory from temporary file
pdl1_model = pdl1_struct[0]                                          # Define first (and only) model in PD-L1 structure


**Compute SASA (Shrake-Rupley)**

The function will take a biopython model as an input and return the solvent accessible surface area per residue and return it as a dictionary

In [ ]:
def per_residue_sasa(model):
    sr = ShrakeRupley()                      # Create ShrakeRupley object to compute SASA
    sr.compute(model, level="A")             # Compute SASA at atom level for the given model
    res_sasa = {}                            # Initialize dictionary to store SASA per residue
    for chain in model:
        for res in chain:
            if res.id[0] != " ":             # Skip heteroatoms / waters
                continue
            sasa = sum(getattr(atom, "sasa", 0.0) for atom in res)           # Sum SASA of all atoms in the residue
            res_sasa[(chain.id, res.id[1], res.id[2].strip() or "")] = sasa  # Store SASA in dictionary with unique residue ID as key
    return res_sasa

sasa_complex = per_residue_sasa(model)       # PD-L1 inside complex
sasa_free    = per_residue_sasa(pdl1_model)  # PD-L1 alone

**Distance based contact check**

In [ ]:
# Build binder atom list (all chains not in PD-L1)
binder_atoms = []                                                             # List to hold all heavy atoms from binder (antibody)
for ch in model:  
    if ch.id not in pdl1_chains:
        for res in ch:
            if res.id[0] != " ": continue
            for atom in res:
                if atom.element != "H":
                    binder_atoms.append(atom)
ns_binder = NeighborSearch(binder_atoms)                                      # Create NeighborSearch object for fast spatial queries

def residue_in_contact_with_binder(residue, ns, cutoff=4.5):                  # True if residue has any heavy atom within cutoff Å of binder atoms
    my_atoms = [a for a in residue if a.element != "H"]                       # List of heavy atoms in the residue
    for a in my_atoms:
        if ns.search(a.coord, cutoff):                                        # Search for any binder atom within cutoff of this atom
            return True
    return False

**Build a DataFrame**

In [ ]:
rows = []
for ch in model:
    if ch.id in pdl1_chains:
        for res in ch:
            if res.id[0] != " ": 
                continue
            rid = residue_id(res)
            asa_c = sasa_complex.get(rid, 0.0)
            asa_f = sasa_free.get(rid, 0.0)
            d_asa = asa_f - asa_c

            # Contact check
            contact_flag = residue_in_contact_with_binder(res, ns_binder, DIST_CUTOFF)

            # Chemical features
            hydro, charge = residue_features(res.get_resname(), PH)

            rows.append({
                "chain": ch.id,
                "resseq": rid[1],
                "icode": rid[2],
                "resname": res.get_resname(),
                "ASA_complex_A2": asa_c,
                "ASA_free_A2": asa_f,
                "dASA_A2": d_asa,
                "is_epitope_dASA": (d_asa >= DSASA_THRESH),
                "is_epitope_distance": contact_flag,
                "is_epitope_intersection": contact_flag and (d_asa >= DSASA_THRESH),
                "hydrophobicity": hydro,
                "charge_at_pH7.4": charge
            })

df = pd.DataFrame(rows).sort_values(["chain","resseq","icode"])


**Output**

In [ ]:
print("\nTop PD-L1 residues by ΔSASA (burial):")
print(df.sort_values("dASA_A2", ascending=False).head(10)[
    ["chain","resseq","resname","dASA_A2","is_epitope_distance"]
])

print("\nIntersection epitope residues (contact + ΔSASA ≥ {:.1f} Å²):".format(DSASA_THRESH))
print(df[df["is_epitope_intersection"]][
    ["chain","resseq","resname","dASA_A2","hydrophobicity","charge_at_pH7.4"]
].to_string(index=False))

df.to_csv(f"{PDB_ID}_epitope_analysis.csv", index=False)
print(f"\nSaved detailed table to {PDB_ID}_epitope_analysis.csv")